# 07: Neural Networks I
Useful resources:
- [Visual Analytics in Deep Learning: An Interrogative Survey for the Next Frontiers](https://arxiv.org/pdf/1801.06889.pdf)
- [Distill](https://distill.pub)

## Imports

In [ ]:
from dataclasses import dataclass, field
from itertools import product
import random

import altair as alt
import numpy as np
import pandas as pd
import pmlb

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.manifold import TSNE

In [ ]:
# If you're running this code locally, this automatically save the chart data in files,
# rather than including the data in the spec. You may need to comment this out on Colab.

!mkdir -p data
alt.data_transformers.enable('json', prefix='data/altair-data')

## Data Preparation and Modeling

Load the mnist dataset and take a random sample of it.

In [ ]:
mnist = pmlb.fetch_data('mnist')

In [ ]:
mnist_small = mnist.sample(n=30000)

Separate the feature values from the target labels. Split the dataset into train and test sets.

In [ ]:
X = mnist_small.drop(columns=['target']).values
y = mnist_small['target'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

Next we'll train a multi-layer perceptron on this dataset.

In [ ]:
nn = MLPClassifier(hidden_layer_sizes=(512, 256))
nn.fit(X_train, y_train)

This model has 4 layers: an input layer, two hidden layers, and an output layer. The hidden layers use the ReLU activation function. The output layer uses the softmax function.

In [ ]:
nn.n_layers_

In [ ]:
relu_df = pd.DataFrame({
    'x': np.arange(-10, 11),
    'y': np.maximum(np.arange(-10, 11), 0)
})

alt.Chart(relu_df).mark_line().encode(
    x='x',
    y=alt.Y('y').title('ReLU(x)'),
)

The `get_layer_output` function below returns the output of the model at a given layer.

References:
- [sklearn source code for generating model predictions](https://github.com/scikit-learn/scikit-learn/blob/f3f51f9b611bf873bd5836748647221480071a87/sklearn/neural_network/_multilayer_perceptron.py#L144).
- [sklearn source code for ReLU and softmax activations](https://github.com/scikit-learn/scikit-learn/blob/f3f51f9b611bf873bd5836748647221480071a87/sklearn/neural_network/_base.py#L47)
- [scipy source code for softmax](https://github.com/scipy/scipy/blob/v1.9.3/scipy/special/_logsumexp.py#L130-L223)

In [ ]:
def relu(X):
    return np.maximum(X, 0)

def softmax(X):
    return np.exp(X) / np.exp(X).sum(axis=1, keepdims=True)

def get_layer_output(model, X, layer):
    output = X
    
    for i in range(layer - 1):
        z = np.dot(output, model.coefs_[i]) + model.intercepts_[i]
        
        if i < model.n_layers_ - 2:
            output = relu(z)
        else:
            output = softmax(z)
        
    return output

For example, we can see that getting the output of the last layer is the same as calling the model's `predict_proba` function.

In [ ]:
nn.predict_proba(X_train[0:3])

In [ ]:
get_layer_output(nn, X_train[0:3], nn.n_layers_)

Let's use t-SNE to create projections of the activations of each layer.

In [ ]:
def embed_activations(X_train, y_train, nn, layers):
    tsne = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3)
    dfs = []
    
    for i in layers:
        activations = get_layer_output(nn, X_train, i)
        embedded = tsne.fit_transform(activations)
        dfs.append(pd.DataFrame({
            'x': embedded[:,0],
            'y': embedded[:,1],
            'layer': i,
            'label': y_train
        }))
    
    return pd.concat(dfs)

In [ ]:
df_embedded = embed_activations(X_train[0:1500], y_train[0:1500], nn, [2, 3, 4])
df_embedded.head()

**Exercise 1:** Create an embedding that compares the activations of the last three layers. This is similar to an approach that is used in the [ActiVis paper](https://arxiv.org/pdf/1704.01942.pdf).

In [ ]:
alt.Chart(df_embedded).mark_circle().encode(
    x='x',
    y='y',
    color='label:N',
    column='layer:N'
)

In our dataset, an image is represented as a flat numpy array of length 784 (28 x 28).

In [ ]:
X_train[0]

We can use Altair to plot this image as a heatmap. The below `get_df` function takes a flat numpy array representing an image as input and returns a pandas dataframe containing the x and y coordinates and value of each pixel in the image.

In [ ]:
def get_df(data):
    indices = np.arange(data.shape[0])
    size = int(np.sqrt(data.shape[0]))
    x = indices % size
    y = np.floor(indices / size)
    
    return pd.DataFrame({
        'x': x,
        'y': y,
        'value': data
    })

In [ ]:
get_df(X_train[0])

**Exercise 2:** 

Finish the `plot_image` function below. The input is the numpy array for an image. The output should be an Altair chart that visualizes the image as a heatmap.

In [ ]:
def plot_image(x):
    return alt.Chart(get_df(x)).mark_rect().encode(
        x=alt.X('x:O').axis(None),
        y=alt.Y('y:O').axis(None),
        color=alt.Color('value').scale(range=['black', 'white'], domain=[0, 255])
    ).properties(
        width=250,
        height=250
    )

In [ ]:
plot_image(X_train[9])

Last week, we covered Shapley values. We learned that Shapley values show how each feature contributes to a prediction. We can use Shapley values to create a [saliency map](https://christophm.github.io/interpretable-ml-book/pixel-attribution.html) for an image.

I've modified the code from last week to work with numpy arrays instead of pandas dataframes.

In [ ]:
'''
X - nd.array containing the entire dataset
x - nd.array containing a single instance
model - trained sklearn model
feature - the index of the feature that we are computing the Shapley value for
iterations - number of iterations to run for
'''
def calculate_shapley_value(X, x, model, feature, label, iterations):
    # keep track of the total from the summation
    value = 0
    
    n_instances, n_features = X.shape
    
    features = list(range(n_features))
    
    # list of features besides the one we are computing the shapley value for
    other_features = features[0:feature] + features[feature + 1:]

    for _ in range(iterations):
        # 1a: get a random instance
        random_instance = X[random.randint(0, n_instances - 1)]
        
        # 1b: select a random set of features
        num_features_to_change = random.randint(0, n_features - 1)
        features_to_change = random.sample(other_features, num_features_to_change)
        
        # 1c: make a copy of the instance x for the randomly selected features,
        # replace the value of that feature in x with the value in random_instance
        z_original = np.copy(x)
        
        for f in features_to_change:
            z_original[f] = random_instance[f]
            
        # 1d: make a copy of z_original. replace the value
        # of feature with the value in random_instance
        z_different = np.copy(z_original)
        z_different[feature] = random_instance[feature]
        
        
        # 1e: get the predicted values for z_original and z_different.
        # calculate the difference between them
        pred_original = model.predict_proba([z_original])[0][label]
        pred_different = model.predict_proba([z_different])[0][label]
        difference = pred_original - pred_different
        
        value += difference
        
    # take the mean
    return value / iterations

The below `shapley_values` function calculates the shapley value of every feature for the instance `x`. It returns a flat numpy array containing the Shapley values for the given instance.

In [ ]:
def shapley_values(X, x, model, label, iterations):
    values = []
    
    for feature in list(range(X.shape[1])):
        values.append(calculate_shapley_value(X, x, model, feature, label, iterations))
            
    return np.array(values)

In [ ]:
saliency = shapley_values(X=X_train, x=X_train[9], model=nn, label=y_train[9], iterations=200)

In [ ]:
saliency

**Exercise 3:** Finish the `plot_saliency` function below. The input is a flat numpy array containing the Shapley values for an instance. This function should return an Altair chart that plots a saliency map.

In [ ]:
def plot_saliency(x):
    return alt.Chart(get_df(x)).mark_rect().encode(
        x=alt.X('x:O').axis(None),
        y=alt.Y('y:O').axis(None),
        color=alt.Color('value').scale(scheme='blueorange', domainMid=0)
    ).properties(
        width=250,
        height=250
    )

In [ ]:
plot_saliency(saliency)

**Exercise 4:** Finish the `plot_image_and_saliency` function below. `image` and `saliency` are both numpy arrays. The function should return an Altair chart that shows the image side-by-side the saliency map.

In [ ]:
def plot_image_and_saliency(image, saliency):
    return (plot_image(image) | plot_saliency(saliency)).resolve_scale(color='independent')

In [ ]:
plot_image_and_saliency(X_train[9], saliency)